# Working with Nested Tables in BigQuery


ISB-CGC Community Notebooks
Check out more notebooks at our [Community Notebooks Repository](https://github.com/isb-cgc/Community-Notebooks)!
```
Title:   Working with Nested Tables in BigQuery
Author:  Lauren Wolfe
Created: 2023-07-17
URL:     https://github.com/isb-cgc/Community-Notebooks/blob/master/Notebooks/How_to_use_nested_tables.ipynb
Purpose: To demonstrate strategies for working with nested table structures in Google BigQuery.
Notes:
```

## Introduction

The data that GDC provides through their API is naturally nested. A given patient may, for instance, have multiple diagnoses at the same time. Frequently, they may receive multiple treatments to combat their cancer or have multiple follow up visits to monitor their cancer's progression. The data generated in these examples means that there will be a variable number of records of each type for a given research subject. Traditionally, ISB-CGC has stored these supplemental records in a separate table. However, they are extracted from GDC in the form of a nested JSON object.

BigQuery supports nested columns, and there are advantages to storing the data in this format. Since all of the data is stored in one table, you don't have to locate and join the supplemental tables in order to access data. Instead, you can use UNNEST clauses to flatten the data as needed for your research. This guide provides an introduction to [using UNNEST in your SQL queries](https://cloud.google.com/bigquery/docs/arrays#querying_nested_arrays).

## Getting Started

Here, we import library modules necessary to run our script, including the bigquery module.

In [ ]:
import time
import datetime
import typing
from google.cloud import bigquery
from google.colab import auth
from google.api_core import client_options
from google.oauth2.credentials import Credentials

# When receiving a BQ result, we can convert each record row to a dict.
# This is the typing definition for the resulting object.
BigQueryRowObject = dict[str, typing.Union[str, bool, int, float, datetime.datetime, None, dict]]

Next we will need to Authorize our access to BigQuery and the Google Cloud. For more information, see '[Quick Start Guide to ISB-CGC](https://https://nbviewer.org/github/isb-cgc/Community-Notebooks/blob/master/Notebooks/Quick_Start_Guide_to_ISB_CGC.ipynb)'. Alternative authentication methods can be found [here](https://googleapis.dev/python/google-api-core/latest/auth.html).

Alternatively, you can run this notebook in 'demo mode' using only a Google Account (i.e. a gmail account), not a billable Google Cloud Account. In 'demo mode',  queries are sent to BigQuery via a CGC proxy, and a CGC account is charged for the queries. However the CGC proxy restricts which queries can be run, and the total amount of BigQuery processing volume (in GB) per user.

In [ ]:
demo_mode = False

if demo_mode:
    !wget -O collab_queries.py https://github.com/isb-cgc/Community-Notebooks/raw/refs/heads/master/BQProxy/collab_queries.py
    from collab_queries import api_endpoint, demo_client_args, demo_job_config_arg

    bigquery.bigquery_magics.context.bigquery_client_options ={"api_endpoint": api_endpoint}
    bigquery.bigquery_magics.context.credentials = Credentials(token='dummy', expiry=None)

else:
    !gcloud auth application-default login

--2025-08-29 20:51:26--  https://github.com/isb-cgc/Community-Notebooks/raw/refs/heads/master/BQProxy/collab_queries.py
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/isb-cgc/Community-Notebooks/refs/heads/master/BQProxy/collab_queries.py [following]
--2025-08-29 20:51:26--  https://raw.githubusercontent.com/isb-cgc/Community-Notebooks/refs/heads/master/BQProxy/collab_queries.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21275 (21K) [text/plain]
Saving to: ‘collab_queries.py’

collab_queries.py   100%[===================>]  20.78K  --.-KB/s    in 0.008s  

2025-08-29 20:51:27 (2.57 MB/s)

Finally, we'll define our billing project. Edit the variable below.

In [ ]:
if demo_mode:
    project_num = "isb-cgc-dev-1"
    location = 'US'

else:
    # Create a variable for which client to use with BigQuery
    project_num = 'your-billing-project-CHANGE-ME' # Update with your Google Project number
    location = 'US'

if project_num == 'your-billing-project-CHANGE-ME':
    print('Please update the project number with your Google Cloud Project')
else:
    client_args = demo_client_args() if demo_mode else {}
    client = bigquery.Client(project_num, **client_args)

## Example 1: Retrieve treatment_ids for a Given Patient

Say that we want to retrieve all of the treatment_ids associated with a given patient. There are multiple ways we could achieve that. One option is to use a BigQuery QueryJob to retrieve the patient record, then use python to locate all the treatment_ids, as shown below:

In [ ]:
#SKIP FOR DEMO MODE. Not yet working. Continue to the next cell
def get_query_result(sql: str, bqparams: list) -> typing.Union[bigquery.table.RowIterator, None]:
  # Executes a given SQL statement and returns the query result.

  # initialize the QueryJob
  job_config = bigquery.QueryJobConfig()
  if bqparams is not None:
      job_config.query_parameters = bqparams


  # execute the query
  query_job = client.query(query=sql, location=location, job_config=job_config)

  while query_job.state != 'DONE':
    query_job = client.get_job(query_job.job_id, location=location)
    # wait for the job to complete
    if query_job.state != 'DONE':
      time.sleep(5)

  query_job = client.get_job(query_job.job_id, location=location)

  if query_job.error_result is not None:
    print(f'[ERROR] {query_job.error_result}')
    return None

  # Return the query result as a RowIterator object
  return query_job.result()


sql = """
  SELECT *
  FROM `isb-cgc-bq.CGCI_versioned.clinical_nested_gdc_r37`
  WHERE case_id = 'c3f876f4-2d3a-4d60-b6c4-019f94010330'
"""
bqparams = [bigquery.ScalarQueryParameter("queryid", "STRING", "nt1"),bigquery.ScalarQueryParameter("case_id", "STRING", "c3f876f4-2d3a-4d60-b6c4-019f94010330")] if demo_mode else None

query_result = get_query_result(sql=sql, bqparams = bqparams)

case_record_list = list()

for case_row in query_result:
  # convert RowIterator object into a list of dictionaries
  case_record = dict(case_row.items())
  case_record_list.append(case_record)

for case_record in case_record_list:
  diagnosis_list = case_record['diagnoses']

  for diagnosis in diagnosis_list:
    treatment_list = diagnosis['treatments']

    for treatment in treatment_list:
      print(treatment['treatment_id'])

KeyError: 'diagnoses'

An alternative, and simpler, approach is to use an UNNEST clause to retrieve the treatment_ids. This example employs the bigquery magic command, which you can learn more about [here](https://notebook.community/GoogleCloudPlatform/python-docs-samples/notebooks/tutorials/bigquery/BigQuery%20query%20magic).

In [ ]:
nt2_params = {"queryid": "nt1", "case_id": "c3f876f4-2d3a-4d60-b6c4-019f94010330"} if demo_mode else {}

In [ ]:
%%bigquery --project $project_num --params $nt2_params
SELECT treatment.treatment_id
FROM `isb-cgc-bq.CGCI_versioned.clinical_nested_gdc_r37`,
UNNEST(diagnoses) AS diagnosis, # first we unnest diagnoses to access its columns
UNNEST(treatments) AS treatment # then we unnest treatments, a child column of diagnoses
WHERE case_id = 'c3f876f4-2d3a-4d60-b6c4-019f94010330'


Query is running:   0%|          |

Downloading:   0%|          |

,disease_type
0,Mature B-Cell Lymphomas


## Example 2: Retrieve submitter_ids from Diagnoses and Treatments

In the query below, we select a patient record using the case_id, then use UNNEST to retrieve the diagnosis and treatment submitter_ids.

Note: diagnoses and treatments both contain a column named submitter_id, which creates a naming conflict when unnested. You can address this by explicitly renaming the columns, as shown in the example below. If you don't, BigQuery will append an integer suffix to every duplicate column name. (e.g. submitter_id, submitter_id_1, etc.)

In [ ]:
nt3_params = {"queryid": "nt2", "case_id": "39dce88d-112c-4a3d-b2d2-11e0616594d8"} if demo_mode else {}

In [ ]:
%%bigquery --project $project_num --params $nt3_params
SELECT diagnosis.submitter_id AS diagnosis_submitter_id,
  treatment.submitter_id AS treatment_submitter_id
FROM `isb-cgc-bq.CGCI_versioned.clinical_nested_gdc_r37` AS base_case,
UNNEST(diagnoses) AS diagnosis
LEFT JOIN UNNEST(diagnosis.treatments) AS treatment
WHERE case_id = '39dce88d-112c-4a3d-b2d2-11e0616594d8'

Query is running:   0%|          |

Downloading:   0%|          |

,treatment_id
0,042b8b49-7a15-477a-9c56-71bb91f3ed3d
1,04eb8fa0-7ca1-4489-a173-5a230f0d51c0
2,17b595a1-50ec-4bcf-a13d-b2e1752e8309
3,19bf9079-280e-40ea-9525-181031578ed6
4,287de6e8-e938-46a1-af93-d3befe9ebd32
5,34449123-341e-592e-84fb-e4899e4249a2
6,4be0f2da-aa39-4846-9fef-8fa9d11a96cf
7,4fb4c310-6859-45b5-955a-fc0f68f876e7
8,636b07e0-1e2d-4887-8d30-98d1de86bd02
9,72559c14-c55e-44f6-8297-6102eb712413


## Example 3: Unnesting Multiple Column Groups

The case we used in the above query doesn't have any follow_up records, so let's look at a different `case_id`: `18395371-3c84-4d39-8ace-a3546e9ea34e`

If we unnest all three nested columns, the resulting output will actually be the Cartesian product, representing every possible combination of the diagnosis, treatment and follow_up records.

The new case has one diagnosis record, 8 treatment records, and 6 follow_up records. $1*8*6=48$. Our result will have 48 rows.

The best way to avoid this is to unnest one set of nested columns at a time, e.g. diagnoses (and optionally its children: diagnoses.treatments, diagnoses.pathlogy_details, diagnoses.annotations).

In [ ]:
nt4_params = {"queryid": "nt4", "case_id": "18395371-3c84-4d39-8ace-a3546e9ea34e"} if demo_mode else {}

In [ ]:
%%bigquery --project $project_num --params $nt4_params
SELECT base_case.case_id,
  diagnosis.diagnosis_id,
  treatment.treatment_id,
  follow_up.follow_up_id
FROM `isb-cgc-bq.CGCI_versioned.clinical_nested_gdc_r37` AS base_case
LEFT JOIN UNNEST(diagnoses) AS diagnosis
LEFT JOIN UNNEST(diagnosis.treatments) AS treatment
LEFT JOIN UNNEST(follow_ups) AS follow_up
WHERE case_id = '18395371-3c84-4d39-8ace-a3546e9ea34e'

Query is running:   0%|          |

Downloading:   0%|          |

,case_id,diagnosis_id,treatment_id,follow_up_id
0,18395371-3c84-4d39-8ace-a3546e9ea34e,20cbc3e8-86f1-43c9-9b6a-d5a85f027685,77f3a3b5-8bf7-4a53-95ed-15079795103e,3eb27fdc-a254-4edd-b271-4062e09b1649
1,18395371-3c84-4d39-8ace-a3546e9ea34e,20cbc3e8-86f1-43c9-9b6a-d5a85f027685,77f3a3b5-8bf7-4a53-95ed-15079795103e,465ed8a1-d0b2-4960-b7e9-611dea4fd952
2,18395371-3c84-4d39-8ace-a3546e9ea34e,20cbc3e8-86f1-43c9-9b6a-d5a85f027685,77f3a3b5-8bf7-4a53-95ed-15079795103e,55d5a520-101b-44e8-80a2-b622b59f42ec
3,18395371-3c84-4d39-8ace-a3546e9ea34e,20cbc3e8-86f1-43c9-9b6a-d5a85f027685,77f3a3b5-8bf7-4a53-95ed-15079795103e,a3d33feb-f2db-4745-8e28-1c2c78093f5f
4,18395371-3c84-4d39-8ace-a3546e9ea34e,20cbc3e8-86f1-43c9-9b6a-d5a85f027685,77f3a3b5-8bf7-4a53-95ed-15079795103e,c41c0549-2d14-4710-91dc-ed0aef3d2e8f
5,18395371-3c84-4d39-8ace-a3546e9ea34e,20cbc3e8-86f1-43c9-9b6a-d5a85f027685,77f3a3b5-8bf7-4a53-95ed-15079795103e,d59e772a-43ea-4ab9-bd02-eef48372ad77
6,18395371-3c84-4d39-8ace-a3546e9ea34e,20cbc3e8-86f1-43c9-9b6a-d5a85f027685,98e3a1df-6ca3-4e2f-a453-ca2da4c3401b,3eb27fdc-a254-4edd-b271-4062e09b1649
7,18395371-3c84-4d39-8ace-a3546e9ea34e,20cbc3e8-86f1-43c9-9b6a-d5a85f027685,98e3a1df-6ca3-4e2f-a453-ca2da4c3401b,465ed8a1-d0b2-4960-b7e9-611dea4fd952
8,18395371-3c84-4d39-8ace-a3546e9ea34e,20cbc3e8-86f1-43c9-9b6a-d5a85f027685,98e3a1df-6ca3-4e2f-a453-ca2da4c3401b,55d5a520-101b-44e8-80a2-b622b59f42ec
9,18395371-3c84-4d39-8ace-a3546e9ea34e,20cbc3e8-86f1-43c9-9b6a-d5a85f027685,98e3a1df-6ca3-4e2f-a453-ca2da4c3401b,a3d33feb-f2db-4745-8e28-1c2c78093f5f
